In [59]:
import yfinance as yf
import datetime as dt
import pandas as pd
# from sklearn.linear_model import LinearRegression
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ruptures as rpt
from ruptures.metrics import precision_recall
import pymannkendall as mk

import os

In [9]:
# Get all SNP500 stock's Closing-price timeseries data
stocks = pd.read_csv('snp500_portfolio.csv')

data = []

# SPY = SNP500 index stock
for ticker in stocks['stock'].tolist() + ['SPY']:
    df = yf.download(tickers = ticker, period = "2y", interval='1d', progress=False).dropna()
    df = df.rename(columns={"Close": "{}".format(ticker)})
    data.append(df["{}".format(ticker)])

# 2 years of data for all stocks
Data = pd.concat(data, axis=1)

### I. Stock Performance Visuals (1 year data)

In [13]:
# 1 year of data for all stocks
data = Data.iloc[Data.shape[0]//2:]

# cumulative returns of all stocks
df = ((((1 + data.pct_change(axis = 0)).cumprod()) - 1) * 100).reset_index().melt(id_vars=["Date"], var_name = 'stock')
df = pd.merge(df, stocks, how='left')
df['Date'] = df['Date'].dt.date

# sector-based cumulative returns
# each sector has only the stocks that are from the SNP500
# Each stock considered for a particular sector is given equal weight
sector_pct_chng = df.groupby(['sector', 'Date']).agg({'value': 'median'}).reset_index()
sector_pct_chng = sector_pct_chng.rename(columns={'sector': 'stock'})

df = pd.concat([df, sector_pct_chng], ignore_index = True)

In [14]:
# Why PolyFit and not other linear estimation algorithms?
# PolyFit is much faster than OLS or LinearRegress
# Any lack of performance it may have in dealing with outliers doesn't matter much if used for preliminary visualization
def getTrend(x):
    z = np.polyfit(np.arange(x.size), x, deg = 1)
    return np.poly1d(z)

In [15]:
fig = make_subplots(rows=1, cols=1, shared_xaxes = True)
fig2 = make_subplots(rows=1, cols=1, shared_xaxes = True)

button_options1 = []
button_options2 = []

for indx, row in stocks.iterrows():
    
    vis1 = [False] * stocks.shape[0] * 2
    vis1[2*indx] = True
    vis1[2*indx + 1] = True

    vis2 = [False] * stocks.shape[0] * 3
    vis2[3*indx] = True
    vis2[3*indx + 1] = True
    vis2[3*indx + 2] = True

    button_options1.append(dict(label = row['stock'], method = 'update', args = [{'visible': vis1}]))
    button_options2.append(dict(label = row['stock'], method = 'update', args = [{'visible': vis2}]))
    
    # Plot cumulative returns of stock for the past 1 year
    fig2.append_trace(
        go.Scatter(
            x = df[df['stock'] == row['stock']]['Date'],
            y = df[df['stock'] == row['stock']]['value'],
            name = "{}".format(row['stock']),
            visible = False),
        row = 1, col = 1
    )

    # Plot cumulative returns of the Industry to which the above stock belonfs
    fig2.append_trace(
        go.Scatter(
            x = df[df['stock'] == row['sector']]['Date'],
            y = df[df['stock'] == row['sector']]['value'],
            name = "{}".format(row['sector']),
            visible = False),
        row = 1, col = 1
    )

    # Plot cumulative returns of SPY
    fig2.append_trace(
        go.Scatter(
            x = df[df['stock'] == "SPY"]['Date'],
            y = df[df['stock'] == "SPY"]['value'],
            name = "SPY",
            visible = False),
        row = 1, col = 1
    )

    # Plot stock price chart
    fig.append_trace(
        go.Scatter(x = Data.index,
        y=Data[row['stock']], name = "{} Observed Closing Price".format(row['stock']),
        visible = False),
        row = 1, col = 1
    )

    # Plot stock's overall trend estimate for the past 1 year
    fig.append_trace(
        go.Scatter(x = Data.index,
        y = getTrend(Data[row['stock']])(np.arange(Data[row['stock']].size)),
        name = "{} Trend".format(row['stock']),
        visible = False),
        row = 1, col = 1
    )

fig.update_layout(
    yaxis=dict(title="USD"),
    title = "Stock Performance TrendLine",
    template = 'plotly_dark',
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list([dict(label = 'None', method = 'update', args = [{'visible': [False] * stocks.shape[0] * 2}])] + button_options1)
        )
    ])

fig2.update_layout(
    template = 'plotly_dark',
    title='Performance - Cumulative Returns using Closing Prices of Daily Chart',
    yaxis=dict(title="Cumulative Returns (%)"),
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list([dict(label = 'None', method = 'update', args = [{'visible': [False] * stocks.shape[0] * 3}])] + button_options2)
        )
    ])

# Plot SPY's price chart
fig1 = make_subplots(rows=1, cols=1, shared_xaxes = True)
fig1.append_trace(
    go.Scatter(x=Data.index,
    y=Data["SPY"], name = "S&P500 Observed Closing Price"),
    row = 1, col = 1
)

# Plot SPY's overall price trend estimate for the past 1 year
fig1.append_trace(
    go.Scatter(x=Data.index, y=getTrend(Data["SPY"])(np.arange(Data["SPY"].size)), name = "S&P500 Trend"),
    row = 1, col = 1
)

fig1.update_layout(yaxis=dict(title="USD"), template = 'plotly_dark')

# Delete output.html if it already exists
if os.path.exists('snp500_stock_performance.html'):
    os.remove('snp500_stock_performance.html')

with open('snp500_stock_performance.html', 'w', encoding = 'utf-8') as f:
    f.write(fig2.to_html())
    f.write(fig.to_html())
    f.write(fig1.to_html())

### II. Trend Nuances

1. If a stock was temporarily high/low/price-shocked anywhere in the time-series, then post such fluctuations/shocks, the trend may:
    1. rebound and get back in sync with its prior trend, Or
    2. remain flat at its new post- shock/fluctuation level, Or
    3. go against its prior trend (i.e. if the stock was increasing, then post shock, it may continue decreasing)
2. If the stock goes temporarily high/low/price-shocked some time before its 85% run (i.e. say, prior to 3 months from now in the past), then Mann-Kendall test would have enough data to classify the trend appropriately. But such shocks/fluctuations happen recently (i.e. assume the market had crashed in the past 3 months), then Mann-Kendall may not catch such nuances.
3. Thus we aim to:
    1. Detect whether such a fluctuation/shock/trend-shift happened in the past 3 months using ChangePoint detection
    2. If ChangePoint is detected, we calculate the post and prior trends using Mann-Kendall test, and then finally
    3. Classify the stock into one of 6 buckets (Increasing), (Inc/Recent Trend Change), (Decreasing), (Dec/RTC), (Static), and
    (Static/RTC)

Before running Mann-Kendall Trend test, we ensure that our timseries data:
1. Isn’t collected seasonally 
2. Does not have any covariates
3. Has only one data point per time period

To show the merits of using Mann-Kendall test for Trend Analysis (see AMZN and TSLA for the past 1 year):
1. If we look solely at these stock's final and initial values, they both show negative returns.
2. See their LinearTrend in the visual. LinearTrend shows TSLA as slightly decreasing and AMZN as increasing
3. But the difference is that LinearTrend tries to draw a straight line that is closest to most points on the timeseries,
while Mann-Kendall looks only at the distribution of Count(+ Change) and Count(- Change) to predict trend
4. Thus, Mann-Kendall predicts TSLA as static, but predicts AMZN as increasing

#### IIa. ChangePoint/Price Shock Detection
1. Detect points on a timeseries that statistically appear to demonstrate a change in the trend.
2. If the latest changepoint is within the past 3 months, detect if its trend is contrary to the overall 2Year trend of the stock.

In [27]:
"""
    CostFunction hyperparameter tuning.
    Use manually labelled changepoint data (see stock_changes_manual_labels.csv) to evaluate performance of each costFunc.
    The model is tested with the data obtained during the COVID stock market crash time window.
"""
# Result = Rank CostFunction seems to be the most optimal costFunc.

stk_lbls = pd.read_csv("stock_changes_manual_labels.csv")
stk_lbls['changepoint'] = stk_lbls["changepoint"].fillna("[]").apply(lambda x: eval(x))

startdate = dt.datetime.strptime("2018-07-01", "%Y-%m-%d").date()
enddate = dt.datetime.strptime("2020-07-01", "%Y-%m-%d").date()

prediction = []

for stk in stk_lbls['stock']:

    ts = yf.download(tickers=stk, start=startdate, end=enddate, interval="1d", progress=False).reset_index()['Close'].values

    for cost_func in ['l1', 'l2', 'normal', 'rbf', 'cosine', 'clinear', 'rank', 'mahalanobis', 'ar']:
        
        algo = rpt.Pelt(model=cost_func, min_size=60)
        algo.fit(ts)
        prediction.append([stk, cost_func, algo.predict(pen=1)])

prediction = pd.DataFrame(prediction, columns = ['stock', 'costfunc', 'pred'])
prediction = pd.merge(prediction, stk_lbls, how = 'left')

precision = []
recall = []
for indx, row in prediction[['pred', 'changepoint']].iterrows():
    try:
        p, r = precision_recall(row['changepoint']+[row['pred'][-1]], row['pred'], margin = 15)
        precision.append(p)
        recall.append(r)
    except:
        precision.append(0)
        recall.append(0)

prediction['precision'] = precision
prediction['recall'] = recall

c:\Users\Jay\AppData\Local\Programs\Python\Python310\lib\site-packages\ruptures\costs\costnormal.py:27: UserWarning:

New behaviour in v1.1.5: a small bias is added to the covariance matrix to cope with truly constant segments (see PR#198).

c:\Users\Jay\AppData\Local\Programs\Python\Python310\lib\site-packages\ruptures\costs\costnormal.py:27: UserWarning:

New behaviour in v1.1.5: a small bias is added to the covariance matrix to cope with truly constant segments (see PR#198).

c:\Users\Jay\AppData\Local\Programs\Python\Python310\lib\site-packages\ruptures\costs\costnormal.py:27: UserWarning:

New behaviour in v1.1.5: a small bias is added to the covariance matrix to cope with truly constant segments (see PR#198).

c:\Users\Jay\AppData\Local\Programs\Python\Python310\lib\site-packages\ruptures\costs\costnormal.py:27: UserWarning:

New behaviour in v1.1.5: a small bias is added to the covariance matrix to cope with truly constant segments (see PR#198).

c:\Users\Jay\AppData\Local\Progr

In [149]:
fig = make_subplots(rows=1, cols=1, shared_xaxes = True)

button_options = []

trend_analysis = []

for indx, row in stocks.iterrows():

    if Data[row['stock']].isnull().any():
        shift_index = Data[row['stock']].reset_index()[row['stock']].first_valid_index()
    else:
        shift_index = 0
    
    algo = rpt.Pelt(model="rank", min_size=60)
    result = algo.fit(Data[row['stock']].dropna().values).predict(pen=1)[-2]
    
    vis = [False] * stocks.shape[0] * 2
    vis[2*indx] = True
    vis[2*indx + 1] = True

    button_options.append(dict(label = row['stock'], method = 'update', args = [{'visible': vis}]))

    # Plot stock price chart
    fig.append_trace(
        go.Scatter(x = Data.index,
        y=Data[row['stock']], name = "{} Observed Closing Price".format(row['stock']),
        visible = False),
        row = 1, col = 1
    )

    # Plot stock's overall trend estimate for the past 1 year
    fig.append_trace(
        go.Scatter(x = [Data.iloc[result - 1 + shift_index].name] * 2,
        y = [0, Data.iloc[result - 1 + shift_index][row['stock']]],
        line = dict(dash='dash'),
        name = "Recent Changepoint",
        visible = False),
        row = 1, col = 1
    )

    # Trend Analysis
    if Data.iloc[result - 1 + shift_index].name > pd.to_datetime(dt.datetime.today().date() - dt.timedelta(days=100)):
        # timeseries trend before change
        mktest_prior = mk.original_test((data[row['stock']][:Data.iloc[result - 1 + shift_index].name].values).reshape(-1, 1))
        mktest_post = mk.original_test((data[row['stock']][Data.iloc[result - 1 + shift_index].name:].values).reshape(-1, 1))

        if mktest_prior.p > 0.05:
            prior_trend = 'no change'
        else:
            prior_trend = mktest_prior.trend
        if mktest_post.p > 0.05:
            post_trend = 'no change'
        else:
            post_trend = mktest_post.trend

        if prior_trend != post_trend:
            trend_analysis.append([row['stock'], prior_trend  + "/" + "RTC", [mktest_prior.p, mktest_post.p]])
        else:
            trend_analysis.append([row['stock'], prior_trend, [mktest.p, mktest_post.p]])
    else:
        mktest = mk.original_test((data[row['stock']].values).reshape(-1, 1))
        if mktest.p < 0.05:
            trend_analysis.append([row['stock'], mktest.trend, [mktest.p, 0]])
        else:
            trend_analysis.append([row['stock'], 'no change', [mktest.p, 0]])

fig.update_layout(
    yaxis=dict(title="USD"),
    title = "ChangePoints",
    template = 'plotly_dark',
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list([dict(label = 'None', method = 'update', args = [{'visible': [False] * stocks.shape[0] * 2}])] + button_options)
        )
    ])

fig1.update_layout(yaxis=dict(title="USD"), template = 'plotly_dark')

# Delete output.html if it already exists
if os.path.exists('changepoints.html'):
    os.remove('changepoints.html')

with open('changepoint.html', 'w', encoding = 'utf-8') as f:
    f.write(fig.to_html())


#### III. P-values quantify Class prediction(s)

In [155]:
trend_analysis = pd.DataFrame(trend_analysis, columns = ['stock', 'class', 'p_values'])
trend_analysis.head()

,stock,class,p_values
0,A,increasing,"[0.8435152912320247, 0.004983598009033363]"
1,AAL,increasing,"[0.8435152912320247, 0.00012088374218888198]"
2,AAP,decreasing/RTC,"[0.0, 0.5985347248798343]"
3,AAPL,increasing,"[0.0, 0]"
4,ABBV,no change,"[0.8435152912320247, 0]"


In [156]:
set(trend_analysis['class'])

{'decreasing',
 'decreasing/RTC',
 'increasing',
 'increasing/RTC',
 'no change',
 'no change/RTC'}

In [163]:
trend_analysis[trend_analysis['stock'] == 'CNC']

,stock,class,p_values
106,CNC,decreasing,"[0.0, 0]"
